In [ ]:
from openai import OpenAI
from key import openai_key
client = OpenAI(api_key=openai_key)



# for b in range(80):

#   batch_input_file = client.files.create(
#     file=open("../../data/batch_request_gpt4omini_"+str(b+1)+".jsonl", "rb"),
#     purpose="batch"
#   )
#   batch_input_file_id = batch_input_file.id

#   metadata = client.batches.create(
#       input_file_id=batch_input_file_id,
#       endpoint="/v1/chat/completions",
#       completion_window="5h",
#       metadata={
#         "description": "version 1.0",
#       }
#   )

#   print(metadata)

b=1
# around 30min to run one batch
batch_input_file = client.files.create(
  file=open("../../data/batch_request_gpt4omini_"+str(b+1)+".jsonl", "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id

metadata = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "version 1.0",
    }
)

print(metadata)


Batch(id='batch_67285e6419c88190abe14329db2a30b6', completion_window='24h', created_at=1730698852, endpoint='/v1/chat/completions', input_file_id='file-zzVeklvTxvTUOMQNGL1PfKiY', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1730785252, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'version 1.0'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


# Wait until it finished, you can run it if you are not sure whether it is finished

In [18]:
batch_id = metadata.id
metadata = client.batches.retrieve(batch_id)
batch_status = metadata.status 
print(batch_status)
output_file_id = metadata.output_file_id

file_response = client.files.content(output_file_id)
output_file_id
# print(file_response.text)

completed


'file-tHvpUnHH9bAVYIlNR24A3RfT'

In [ ]:
import json 
ids = []
responses = []
for data in file_response.iter_lines():
    # print(data)
    output = json.loads(data)
    
    # print(output['custom_id'], output['response']['body']['choices'][0]['message']['content'])
    ids.append(output['custom_id'])
    responses.append(output['response']['body']['choices'][0]['message']['content'])

import pandas as pd
df = pd.DataFrame({"sm_id": ids,"responses": responses})

# Write to CSV while preserving line breaks
csv_file_path = "../../gpt_data/output"+str(b+1)+".csv"
df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL
print(f"CSV file created at {csv_file_path}")


CSV file created at ../../gpt_data/output2.csv


In [20]:
read_df = pd.read_csv(csv_file_path)
read_df.head()

,sm_id,responses
0,111334d,"(1) relation: [no], related phrases if any: \..."
1,1115m19,"(1) relation: [yes], related phrases if any: r..."
2,11161im,"(1) relation: [yes], related phrases if any: ""..."
3,1116b9m,"(1) relation: yes, related phrases if any: fam..."
4,11174df,"(1) relation: [no], related phrases if any: \..."


# continue processing batch by batch

In [ ]:
import json 
import os
import time
import pandas as pd
from openai import OpenAI
from key import openai_key
client = OpenAI(api_key=openai_key)

for b in range(80):

  csv_file_path = "../../gpt_data/output"+str(b+1)+".csv"

  # Check if the CSV file already exists
  if not os.path.exists(csv_file_path):
    print( " ----- start processing " + str(b+1) + " ----- ")

    batch_input_file = client.files.create(
      file=open("../../data/batch_request_gpt4omini_"+str(b+1)+".jsonl", "rb"),
      purpose="batch"
    )
    batch_input_file_id = batch_input_file.id
    metadata = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
          "description": "version 1.0",
        }
    )


    batch_id = metadata.id
    batch_status = metadata.status

    # Poll every 5 minutes for batch status until completed
    while batch_status != "completed":
        time.sleep(60*5)  # Sleep for 5 minutes 
        
        # Retrieve the latest metadata to check the status
        metadata = client.batches.retrieve(batch_id)
        batch_status = metadata.status
        print(f"Batch {b + 1} status: {batch_status}")
        
        if batch_status == "completed":
            print(f"Batch {b + 1} has completed.")
            break
        

    output_file_id = metadata.output_file_id
    file_response = client.files.content(output_file_id)
    ids = []
    responses = []
    for data in file_response.iter_lines():
        output = json.loads(data)
        ids.append(output['custom_id'])
        responses.append(output['response']['body']['choices'][0]['message']['content'])

    
    df = pd.DataFrame({"sm_id": ids,"responses": responses})
    # Write to CSV while preserving line breaks
    df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL
    print(f"CSV file created at {csv_file_path}")


 ----- start processing 3 ----- 
